In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

## Implement the Ensemble methods learnt in class and compare their accuarcies.

The dataset you are going to be using for homework is the **Wisconsin Breast Cancer dataset (cancer.csv)**

The dataset contains a total number of 10 features labeled in either benign or malignant classes. The features have 699 instances out of which 16 feature values are missing. The dataset only contains numeric values.

Attribute Information:

1. Sample code number: id number
2. Clump Thickness: 1 - 10
3. Uniformity of Cell Size: 1 - 10
4. Uniformity of Cell Shape: 1 - 10
5. Marginal Adhesion: 1 - 10
6. Single Epithelial Cell Size: 1 - 10
7. Bare Nuclei: 1 - 10
8. Bland Chromatin: 1 - 10
9. Normal Nucleoli: 1 - 10
10. Mitoses: 1 - 10
11. Class: (2 for benign, 4 for malignant) (**target variable**)

For more information: https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)

### 1. Read the dataset into variable called '**data**' (1 mark)

In [ ]:
pd.set_option('display.max_columns', 100)
data = pd.read_csv('/content/cancer(3).csv')

### **Preprocessing**: Data needs to be preprocessed before implementing ensemble methods. It is done for you here. 
### Run the below code first and then answer the questions from 2 - 7.

#### Deleting unnecessary columns: The column "Sample code number" is just an indicator and it's of no use in the modeling. So, let's drop it:


In [ ]:
data.drop(['Sample Code Number'],axis = 1, inplace = True)

#### Handling missing values : 
As mentioned earlier, the dataset contains missing values. The column named "Bland Chromatin" contains them. The missing values are represneted as "?". 

Replace those "?"s with 0's and impute them with Mean Imputation

In [ ]:
data['Bland Chromatin']

0       1
1      10
2       2
3       4
4       1
       ..
694     2
695     1
696     3
697     4
698     5
Name: Bland Chromatin, Length: 699, dtype: object

In [ ]:
data.replace('?',0, inplace=True)

In [ ]:
# Convert the DataFrame object into NumPy array otherwise you will not be able to impute
values = data.values
# Now impute it
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputedData = imputer.fit_transform(values)

#### Normalizing the data:
Ranges of the features of the dataset are not the same. This may cause a problem. A small change in a feature might not affect the other. To address this problem, normalize the ranges of the features to a uniform range, in this case, 0 - 1.

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
normalizedData = scaler.fit_transform(imputedData)
cols = ['Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bland Chromatin', 'Bare Nuclei', 'Normal Nucleoli', 'Mitosis','Class']
normalizedData = pd.DataFrame(normalizedData, columns=cols)
print(normalizedData.head())

   Clump Thickness  Uniformity of Cell Size  Uniformity of Cell Shape  \
0         0.444444                 0.000000                  0.000000   
1         0.444444                 0.333333                  0.333333   
2         0.222222                 0.000000                  0.000000   
3         0.555556                 0.777778                  0.777778   
4         0.333333                 0.000000                  0.000000   

   Marginal Adhesion  Single Epithelial Cell Size  Bland Chromatin  \
0           0.000000                     0.111111              0.1   
1           0.444444                     0.666667              1.0   
2           0.000000                     0.111111              0.2   
3           0.000000                     0.222222              0.4   
4           0.222222                     0.111111              0.1   

   Bare Nuclei  Normal Nucleoli  Mitosis  Class  
0     0.222222         0.000000      0.0    0.0  
1     0.222222         0.111111      0.0

### Data preprocessing is done and now you will answer the below questions using the **normalizedData**: 

### 2. Split the data into test and training data with test size - 30%. Compute the baseline classification accuracy for X_train. (3 marks)

In [ ]:
X = normalizedData.iloc[:, 0:9]
y = normalizedData.iloc[:, 9]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2020, stratify=y)

from sklearn.dummy import DummyClassifier
dummy_classifier = DummyClassifier(strategy='most_frequent')
dummy_classifier.fit(X_train,y_train)
baseline_acc = dummy_classifier.score(X_test,y_test)

print("Baseline Accuracy = ", baseline_acc)

Baseline Accuracy =  0.6571428571428571


### 3.  Bagging : Build a generic Bagging ensemble and print the accuracy (4 marks)
---


Hyperparameters:

Base estimator = DecisionTreeClassifier

n_estimators = 10

random_state = 42

---


In [ ]:
# Generic Bagging model
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

base_est1 = DecisionTreeClassifier(max_depth =4)

model_bagging = BaggingClassifier(base_est1, n_estimators = 10, random_state = 42)
model_bagging.fit(X_train, y_train)
pred_bagging = model_bagging.predict(X_test)
print(pred_bagging)
acc_bagging = accuracy_score(y_test, pred_bagging)

print('Accuracy = ', acc_bagging)

### 4. RandomForest : (7 marks)
#### a) Build a Random Forest model and print the accuracy (4 marks)
---

Constructor arguments: 


n_estimators = 100, max_features = 7 and random_state = 42 


---




In [ ]:
# Random Forest model
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=100, max_features=7, random_state=42)
model_rf.fit(X_train, y_train)
predict_rf = model_rf.predict(X_test)
acc_rf = accuracy_score(y_test, predict_rf)
print('Accuracy = ', acc_rf)

Accuracy =  0.9380952380952381


####  b) Calculate the top 3 important features for the above **RandomForest** model and print them (3 marks)

In [ ]:
# Top 3 features for RandomForest
importance = pd.DataFrame(zip(X_train.columns, model_rf.feature_importances_))
importance.columns=['Feature','Importance']
importance_sorted = importance.sort_values(by=['Importance'], ascending = False)
top3 = importance_sorted.iloc[0:3,:]
print(top3)

                    Feature  Importance
1   Uniformity of Cell Size    0.539650
2  Uniformity of Cell Shape    0.178747
5           Bland Chromatin    0.145336


### 5. Boosting: (7 marks)
#### a) Build an AdaBoost model with training data and print the accuracy (4 marks)
---

Hyperparameters:

Base estimator = DecisionTreeClassifier, max_depth = 4

n_estimators = 200

random_state = 42

learning_rate = 0.05


---









In [ ]:
# AdaBoost Classification
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
base_est2 = DecisionTreeClassifier (max_depth =4)
ada_boost = AdaBoostClassifier(base_est2, n_estimators=200, random_state=42, learning_rate=0.05)
ada_boost.fit(X_train, y_train)
predict_ada = ada_boost.predict(X_test)
acc_ada = accuracy_score(y_test, predict_ada)
print('Accuracy score',acc_ada)

Accuracy score 0.9476190476190476


#### b) Calculate the top 3 important features for the above **AdaBoost** model and print them (3 marks)

In [ ]:
# Top 3 features for AdaBoost
# Enter your code here
imp_ada = pd.DataFrame(zip(X_train.columns, ada_boost.feature_importances_))
imp_ada.columns=['Feature','Importance']
imp_ada_sorted = imp_ada.sort_values(by=['Importance'], ascending = False)
top3_ada = imp_ada_sorted.iloc[0:3,:]
print(top3_ada)

             Feature  Importance
3  Marginal Adhesion    0.237847
0    Clump Thickness    0.211908
6        Bare Nuclei    0.162136


### 6. Voting : Using a voting classifier, build an ensemble of RandomForestClassifier, DecisionTreeClassifier, Support Vector Machine and Logistic Regression. (7 marks)


---


Use max_depth = 4, n_estimators = 200, voting = soft

In [ ]:
# Voting Ensemble for Classification
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

rf = RandomForestClassifier(n_estimators=200, max_depth=4, random_state=42)
dt = DecisionTreeClassifier(max_depth = 4, random_state=42)
svm = SVC(probability=True, random_state=42)
log = LogisticRegression(random_state=42)

clf2 = VotingClassifier(estimators = [('rf',rf), ('dt', dt), ('svm',svm), ('log', log)], voting='soft')
clf2.fit(X_train, y_train)
clf2_pred = clf2.predict(X_test)
print('Accuracy score', accuracy_score(y_test, clf2_pred))

Accuracy score 0.9428571428571428


### 7. Mention the best model among the above 4 models and its accuracy (1 mark)

In [ ]:
# Write your answer here
print("Both Bagging model and AdaBoost model have the best performance with a same accuracy score of 0.9476190476190476")

Both Bagging model and AdaBoost model have the best performance with a same accuracy score of 0.9476190476190476
